## Load Dependencies


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import importlib
import xgboost as xgb
from tqdm import tqdm
from collections import defaultdict
from model_utils import *
from ModelInfo import ModelInfo
import traceback
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import all_estimators
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.datasets import make_regression
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.multioutput import MultiOutputRegressor

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Chargement des modèles sklearn


L'objectif étant de comparer les performances de différents modèles, nous proposons une approche comparant tous les modèles régressif de la bibliothèque scikit-learn, ainsi que d'autres modèles souvent utilisés pour de tâches de régression (ex: xgboost). \
/!\ La comparaison de ces modèles n'a de sens que si l'on comprend le fonctionnement interne de chacun des modèles de la librairie. \
L'approche exploratoire proposée ici est un challenge ajouté au projet. Nous ne retiendrons que les résultats des modèles les plus connus pour répondre à la problématique.


### 1. Récupération des modèles régression de sklearn - classement par groupes custom


In [ ]:
all_estimators_list = all_estimators(type_filter='regressor')

# catégories de modèles
model_categories = {
    "Linear Models": [],
    "Tree-based Models": [],
    "Ensemble Methods": [],
    "Bayesian Models": [],
    "Support Vector Machines": [],
    "K-Nearest Neighbors": [],
    "Neural Networks": [],
    "Others": []
}

for name, EstimatorClass in all_estimators_list:
    try:
        model_instance = EstimatorClass()
        model_description = EstimatorClass.__doc__.split("\n")[0] if EstimatorClass.__doc__ else "No description"
    except TypeError:
        model_description = "Requires additional arguments"

    # On filtre par catégories de modèles
    if issubclass(EstimatorClass, LinearRegression):
        model_categories["Linear Models"].append((name, model_description))
    elif issubclass(EstimatorClass, DecisionTreeRegressor):
        model_categories["Tree-based Models"].append((name, model_description))
    elif issubclass(EstimatorClass, RandomForestRegressor):
        model_categories["Ensemble Methods"].append((name, model_description))
    elif 'Bayes' in name:
        model_categories["Bayesian Models"].append((name, model_description))
    elif 'SVR' in name:
        model_categories["Support Vector Machines"].append((name, model_description))
    elif 'KNeighbors' in name:
        model_categories["K-Nearest Neighbors"].append((name, model_description))
    elif 'MLP' in name:
        model_categories["Neural Networks"].append((name, model_description))
    else:
        model_categories["Others"].append((name, model_description))

# modèles + descriptions
for category, models in model_categories.items():
    print(f"\n{category}:")
    for model_name, description in models:
        print(f" - {model_name}: {description}")


### 2. Récupération des modèles régression de sklearn - classement par modules sklearn


In [ ]:
all_estimators_list = all_estimators(type_filter='regressor')
regressor_models_by_module = defaultdict(list)

for name, EstimatorClass in all_estimators_list:
    try:
        model_instance = EstimatorClass()
        model_description = EstimatorClass.__doc__.split("\n")[0] if EstimatorClass.__doc__ else "No description"
    except TypeError:
        # Si l'initialisation échoue = nécessite des arguments supplémentaires
        model_description = "Requires additional arguments"
    
    module = EstimatorClass.__module__.split('.')[1]  # ex: 'linear_model', 'tree'
    regressor_models_by_module[module].append((name, EstimatorClass, model_description))


def print_regressor_models_by_module(models_by_module):
    for module, models in models_by_module.items():
        print(f"\nModule: {module}")
        for model_name, _, description in models:
            print(f" - {model_name}: {description}")

print_regressor_models_by_module(regressor_models_by_module)


### 3. Test de séparation des modèles supportant le multi-output


In [ ]:
# Jeu de données de régression avec plusieurs sorties (multi-output)
X, y = make_regression(n_samples=100, n_features=4, n_targets=2, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

all_estimators_list = all_estimators(type_filter='regressor')


regressor_models_by_module = defaultdict(list)
multioutput_models = defaultdict(list)
non_multioutput_models = defaultdict(list)


for name, EstimatorClass in all_estimators_list:
    try:
        model_instance = EstimatorClass()
        model_description = EstimatorClass.__doc__.split("\n")[0] if EstimatorClass.__doc__ else "No description"
        model_instance.fit(X_train, y_train) # Multi-output
        # Si l'entraînement réussit, on considère que le modèle supporte le multi-output
        multioutput_models[EstimatorClass.__module__.split('.')[1]].append((name, EstimatorClass, model_description))
    except (TypeError, ValueError):
        # Sinon, des arguments sont nécessaires ou le modèle ne supporte pas le multi-output
        non_multioutput_models[EstimatorClass.__module__.split('.')[1]].append((name, EstimatorClass, model_description))
        continue


print("\n=== Models that natively support multi-output ===")
print_regressor_models_by_module(multioutput_models)

print("\n=== Models that do not support multi-output ===")
print_regressor_models_by_module(non_multioutput_models)


### 4. Chargement des modèles sklearn + externes via la classe ModelInfo


In [ ]:
# Utilisation de la classe custom ModelInfo avec un modèle externe comme XGBoost
external_models = [
    ('XGBRegressor', xgb.XGBRegressor)
]

model_info = ModelInfo(external_models=external_models)
model_info.fill_dataframe()

model_info.df_models

In [ ]:
data = model_info.df_models.to_dict('records')

df_models = pd.DataFrame(data)
bool_columns = df_models.select_dtypes(include=[bool])

plt.figure(figsize=(10, 16))
sns.heatmap(bool_columns, annot=True, cmap='coolwarm', cbar=False, yticklabels=df_models['model_name'])
plt.title("Représentation des modèles et de leurs propriétés booléennes")
plt.xlabel("Propriétés")
plt.ylabel("Modèles")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Test des modèles


In [ ]:
df = pd.read_csv('../data/Quality_group.csv')
len(df)

In [ ]:
# X, y = make_regression(n_samples=100, n_features=4, n_targets=2, noise=0.1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# load the dataset Quality_group.csv
df = pd.read_csv('../data/Quality_group.csv')
target_labels = ['Yield strength / MPa', 
                 'Ultimate tensile strength / MPa', 
                 'Elongation / %', 
                 'Reduction of Area / %']
# target_labels = ['Charpy impact toughness / J',
#                  'Charpy temperature']


X,y = df.drop(target_labels, axis=1), df[target_labels]

# On retire les colonnes one-hot encoded pour le PCA
one_hot_encoded_cols = [col for col in X.columns if X[col].nunique() <= 2 and set(X[col].unique()).issubset({0, 1})]
non_one_hot_cols = X.drop(columns=one_hot_encoded_cols).select_dtypes(include=[np.number]).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[non_one_hot_cols])
X_test_scaled = scaler.transform(X_test[non_one_hot_cols])

# PCA
pca = PCA(n_components=3)  
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PCA_{i+1}' for i in range(X_train_pca.shape[1])])
X_test_pca_df = pd.DataFrame(X_test_pca, columns=[f'PCA_{i+1}' for i in range(X_test_pca.shape[1])])
X_train_final = pd.concat([X_train.reset_index(drop=True), X_train_pca_df], axis=1)
X_test_final = pd.concat([X_test.reset_index(drop=True), X_test_pca_df], axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# # standardize the data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# pca = PCA(n_components=3) 
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_test_pca = pca.transform(X_test_scaled)




def perform_random_grid_search(model, param_distributions, X, y, n_iter=10, cv=3, scoring='neg_mean_squared_error'):
    """
    Effectue une recherche aléatoire sur une grille de paramètres pour un modèle donné.
    Args:
        model: Estimator object.
        param_distributions: Dictionary with parameters names (string) as keys and distributions or lists of parameters to try.
        X: Training data.
        y: Target variable.
        n_iter: Number of parameter settings that are sampled.
        cv: Cross-validation splitting strategy.
        scoring: Scoring method.
    """
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=n_iter, cv=cv, scoring=scoring, random_state=42)
    random_search.fit(X, y)
    return random_search.best_estimator_

    
def train_and_evaluate_models_cv(df_models, X, y, random_grid_search=False, param_grids=None, verbose=False):
    """
    Entraîne et évalue des modèles de régression avec validation croisée.
    Args:
        df_models: DataFrame with models information.
        X: Training data.
        y: Target variable.
        random_grid_search: Perform random grid search for hyperparameter tuning.
        param_grids: Dictionary with hyperparameter grids for each model.
        verbose: Print additional information.
    Returns:
        DataFrame with evaluation results.
    """
    results = []
    
    # Avantages de make_scorer:
    # - Permet de personnaliser les métriques de scoring
    # - Permet de spécifier si une métrique doit être maximisée ou minimisée
    # - cross_validate et GridSearchCV prennent en charge les objets make_scorer
    scoring = {
        'MSE': make_scorer(mean_squared_error),
        'MAE': make_scorer(mean_absolute_error),
        'R2': make_scorer(r2_score)
    }
    
    for idx, row in tqdm(df_models.iterrows(), total=df_models.shape[0], desc="Training Models"):
        try:
            name = row['model_name']
            EstimatorClass = row['estimator']  # Référence à la classe de l'estimateur
            multi_output_native = row['multi_output_native']
            multi_output_compatible = row['multi_output']
            
            # modèle natif multi-output
            if multi_output_native:
                model_instance = EstimatorClass()
            # modèle compatible avec MultiOutputRegressor
            elif multi_output_compatible:
                model_instance = MultiOutputRegressor(EstimatorClass())
            else:
                continue
            
            # Grid search
            if random_grid_search and param_grids and name in param_grids:
                tqdm.write(f"Performing Random Grid Search for {name}...")
                model_instance = perform_random_grid_search(model_instance, param_grids[name], X, y)
            
            # Validation croisée
            cv_results = cross_validate(model_instance, X, y, cv=5, scoring=scoring, return_train_score=True)
            
            # Scores
            train_mse = np.mean(cv_results['train_MSE'])
            val_mse = np.mean(cv_results['test_MSE'])
            train_mae = np.mean(cv_results['train_MAE'])
            val_mae = np.mean(cv_results['test_MAE'])
            train_r2 = np.mean(cv_results['train_R2'])
            val_r2 = np.mean(cv_results['test_R2'])
            
            results.append({
                'Model': name,
                'Description': row['description'],
                'Train MSE': train_mse,
                'Validation MSE': val_mse,
                'Train MAE': train_mae,
                'Validation MAE': val_mae,
                'Train R2': train_r2,
                'Validation R2': val_r2
            })

            if name == 'XGBRegressor':
                model_instance.fit(X, y)
                feature_importances = model_instance.feature_importances_
                feature_names = X.columns
                feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
                feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

                # On garde les 15 premières features et on regroupe les autres dans 'Others'
                top_15 = feature_importances_df.head(15)
                others = pd.DataFrame({
                    'Feature': ['Others ({})'.format(len(feature_importances_df) - 15)], 
                    'Importance': [feature_importances_df.iloc[15:]['Importance'].sum()]
                })
                final_df = pd.concat([top_15, others])

                plt.figure(figsize=(10, 6))
                sns.barplot(x='Importance', y='Feature', data=final_df, palette='rocket')
                plt.title(f"Feature Importance - {name}")
                plt.show()


                
                
            
        except Exception as e:
            tqdm.write(f"Failed to evaluate model {name}:")
            if verbose:
                tqdm.write(str(e))
            # traceback.print_exc()  # Pour le détail des erreurs
            continue
    
    # Retourner les résultats sous forme de DataFrame
    return pd.DataFrame(results)

# Exemple de grille de paramètres pour certains modèles (pour Randomized Grid Search)
# --------------------------
# Pour l'instant, le grid search n'est pas randomisé, on donne juste des valeurs à tester
# --------------------------
param_grids = {
    'RandomForestRegressor': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    'GradientBoostingRegressor': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    },
    'DecisionTreeRegressor': {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    }
}

df_results = train_and_evaluate_models_cv(df_models, X_train_final, y_train, random_grid_search=True, param_grids=param_grids)
df_results = df_results.sort_values(by='Validation R2', ascending=False)

## Analyse des résultats


In [ ]:
def plot_results_heatmap(df_results):
    plt.figure(figsize=(10, 6))
    heatmap_data = df_results[['Model', 'Validation MSE', 'Validation MAE', 'Validation R2']].set_index('Model')
    sns.heatmap(heatmap_data, annot=True, cmap='rocket', cbar_kws={'label': 'Performance'}, fmt='.3f')


    plt.title('Performance des modèles triée par Validation R2', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

plot_results_heatmap(df_results)

def plot_results_barh(df_results, metric='Validation R2', ylim=None):
    plt.figure(figsize=(10, 6))
    df_results = df_results.sort_values(by=metric, ascending=False)
    sns.barplot(x=metric, y='Model', data=df_results, palette='rocket')
    plt.title(f'Modèles triés par {metric}', fontsize=14)
    plt.xlabel(metric)
    plt.ylabel('Modèle')
    # set lims
    if ylim:
        plt.xlim(ylim)
    plt.tight_layout()
    plt.show()


plot_results_barh(df_results, metric='Validation R2', ylim=(0, 1))
